In [1]:
!pip install openai


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# !pip install mysql-connector-python

# 최종

In [5]:
import openai
import mysql.connector
import json

# OpenAI API 키 설정


# MySQL 데이터베이스에 연결
def get_connection():
    return mysql.connector.connect(
        host='localhost',
        user='user',
        password='1234',
        database='finset'
    )

# 사용자의 쿼리로부터 카테고리를 추출
def extract_categories(query):
    response = openai.chat.completions.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": "사용자의 질문에서 금융상품 카테고리와 그에 해당하는 세부 조건(금액, 기간, 이자 방식 등)을 추출하는 시스템입니다. 카테고리는 예금, 적금, 주식, 펀드, 외환입니다."},
            {"role": "user", "content": query}
        ],
        functions=[
            {
                "name": "extract_categories_function",
                "description": "사용자의 질문에서 금융 카테고리와 조건을 추출",
                "parameters": {
                    "type": "object",
                    "properties": {
                         "category": {
                            "type": "string",
                            "description": "추출된 금융 카테고리"
                        },
                        "conditions": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "추출된 세부 조건들(금액, 기간, 이자 방식 등)"
                        }
                    },
                    "required": ["category", "conditions"]
                }
            }
        ],
        function_call={"name": "extract_categories_function"}
    )
    
    categories_conditions = response.choices[0].message.function_call.arguments
    return json.loads(categories_conditions)

# 카테고리와 조건에 맞는 데이터를 데이터베이스에서 추출
def fetch_data_by_category_and_conditions(category, conditions):
    connection = get_connection()
    cursor = connection.cursor(dictionary=True)
    
    # 카테고리별 테이블과 해당 컬럼 결정
    table = ""
    price_column = ""
    interest_column = ""
    
    if category.lower() == '주식':
        table = 'tbl_stock'
        price_column = 'stock_price'
    elif category.lower() == '펀드':
        table = 'tbl_fund'
        price_column = 'fund_standard'
    elif category.lower() == '예금':
        table = 'tbl_deposit'
        interest_column = 'deposit_category'
    elif category.lower() == '적금':
        table = 'tbl_installment'
        interest_column = 'installment_category'
    elif category.lower() == '외환':
        table = 'tbl_exchange'
        price_column = 'exchange_buy'

    # 조건 기반으로 쿼리 생성
    where_clauses = []
    params = []
    
    for condition in conditions:
        if "원" in condition:  # 금액 조건일 때
            amount = int(condition.replace("원", "").replace(",", "").strip())
            if price_column:
                where_clauses.append(f"{price_column} <= %s")
                params.append(amount)
        elif "단리" in condition or "복리" in condition:  # 이자 방식일 때
            if interest_column:
                where_clauses.append(f"{interest_column} LIKE %s")
                params.append(f"%{condition}%")
#         elif "개월" in condition or "년" in condition:  # 기간 조건일 때
#             period = int(''.join(filter(str.isdigit, condition)))
#             where_clauses.append("duration <= %s")
#             params.append(period)

    # 조건이 있을 경우 WHERE 절 추가
    query = f"SELECT * FROM {table}"
    if where_clauses:
        query += " WHERE " + " AND ".join(where_clauses)
    
    cursor.execute(query, tuple(params))
    results = cursor.fetchall()
    connection.close()
    return results

## 예시 1 (주식)

In [3]:
# 사용자 쿼리 예시
query = "100000원에 맞는 주식을 추천해줘"

# 카테고리와 세부 조건을 추출
extracted_data = extract_categories(query)
category = extracted_data['category']
conditions = extracted_data['conditions']

# 카테고리와 조건에 맞는 데이터 추출
relevant_data = fetch_data_by_category_and_conditions(category, conditions)
print(relevant_data)

[{'sno': 1, 'stock_symbol': '005930', 'stock_name': '삼성전자', 'stock_price': '65200', 'price_change_rate': '-1.66', 'stock_volume': '8175458'}, {'sno': 6, 'stock_symbol': '005935', 'stock_name': '삼성전자우', 'stock_price': '53000', 'price_change_rate': '-0.75', 'stock_volume': '622801'}, {'sno': 9, 'stock_symbol': '105560', 'stock_name': 'KB금융', 'stock_price': '80700', 'price_change_rate': '3.33', 'stock_volume': '587431'}, {'sno': 11, 'stock_symbol': '055550', 'stock_name': '신한지주', 'stock_price': '55400', 'price_change_rate': '2.78', 'stock_volume': '330104'}, {'sno': 17, 'stock_symbol': '032830', 'stock_name': '삼성생명', 'stock_price': '96400', 'price_change_rate': '0.21', 'stock_volume': '49934'}, {'sno': 19, 'stock_symbol': '138040', 'stock_name': '메리츠금융지주', 'stock_price': '92400', 'price_change_rate': '2.33', 'stock_volume': '70403'}, {'sno': 20, 'stock_symbol': '086790', 'stock_name': '하나금융지주', 'stock_price': '60800', 'price_change_rate': '4.29', 'stock_volume': '384681'}, {'sno': 25, 'st

In [4]:
print(category)
print(conditions)

주식
['100000원']


In [5]:
import pandas as pd


# relevant_data를 DataFrame으로 변환
df_relevant_data = pd.DataFrame(relevant_data)

# DataFrame 출력
print(df_relevant_data)

   sno stock_symbol stock_name stock_price price_change_rate stock_volume
0    1       005930       삼성전자       65200             -1.66      8175458
1    6       005935      삼성전자우       53000             -0.75       622801
2    9       105560       KB금융       80700              3.33       587431
3   11       055550       신한지주       55400              2.78       330104
4   17       032830       삼성생명       96400              0.21        49934
5   19       138040    메리츠금융지주       92400              2.33        70403
6   20       086790     하나금융지주       60800              4.29       384681
7   25       035720        카카오       35800              1.27       281308
8   30       015760       한국전력       22050              0.46       352370


## 예시 2 (예금)

In [39]:
# 사용자 쿼리 예시
query = "단리 예금 상품 추천해줘"

# 카테고리와 세부 조건을 추출
extracted_data = extract_categories(query)
category = extracted_data['category']
conditions = extracted_data['conditions']

# 카테고리와 조건에 맞는 데이터 추출
relevant_data = fetch_data_by_category_and_conditions(category, conditions)
print(relevant_data)

[{'dno': 1, 'deposit_category': '복리/단리', 'depoist_name': 'e-그린세이브예금', 'deposit_bank': '우한국스탠다드차타드은행', 'deposit_max_rate': '3.37', 'deposit_normal_rate': '3.37', 'deposit_join': '1', 'deposit_stream': '3', 'deposit_amount': '1000000000', 'deposit_way': '인터넷,스마트폰', 'deposit_target': '개인(개인사업자 포함)', 'deposit_link': '직접입력'}, {'dno': 2, 'deposit_category': '단리', 'depoist_name': '스마트정기예금', 'deposit_bank': '신한은행', 'deposit_max_rate': '2.95', 'deposit_normal_rate': '2.95', 'deposit_join': '2', 'deposit_stream': '4', 'deposit_amount': '500000000', 'deposit_way': '인터넷', 'deposit_target': '개인', 'deposit_link': '자동입력'}, {'dno': 4, 'deposit_category': '단리', 'depoist_name': '특별정기예금', 'deposit_bank': '우리은행', 'deposit_max_rate': '2.80', 'deposit_normal_rate': '2.80', 'deposit_join': '3', 'deposit_stream': '5', 'deposit_amount': '1500000000', 'deposit_way': '인터넷', 'deposit_target': '개인', 'deposit_link': '자동입력'}, {'dno': 5, 'deposit_category': '복리/단리', 'depoist_name': '온라인정기예금', 'deposit_bank': '하나은행', 

In [38]:
print(category)
print(conditions)

예금
['단리']


In [40]:
import pandas as pd


# relevant_data를 DataFrame으로 변환
df_relevant_data = pd.DataFrame(relevant_data)

# DataFrame 출력
print(df_relevant_data)

    dno deposit_category depoist_name  deposit_bank deposit_max_rate  \
0     1            복리/단리    e-그린세이브예금  우한국스탠다드차타드은행             3.37   
1     2               단리      스마트정기예금          신한은행             2.95   
2     4               단리       특별정기예금          우리은행             2.80   
3     5            복리/단리      온라인정기예금          하나은행             3.25   
4     7               단리         정기예금          기업은행             2.70   
5     9               단리         정기적금          대구은행             2.85   
6    10            복리/단리       e-정기예금          광주은행             3.20   
7    11               단리       하이정기예금          전북은행             2.75   
8    13               단리      스마트정기예금          강원은행             2.90   
9    15               단리       단기정기예금       BNK부산은행             2.65   
10   16            복리/단리       e-뱅킹예금        하나금융그룹             3.35   
11   17               단리         정기예금          경남은행             2.80   
12   19               단리       우대정기예금         새마을금고             

## 예시 3 (적금)

In [43]:
# 사용자 쿼리 예시
query = "복리 적금 상품은 뭐가있어?"

# 카테고리와 세부 조건을 추출
extracted_data = extract_categories(query)
category = extracted_data['category']
conditions = extracted_data['conditions']

# 카테고리와 조건에 맞는 데이터 추출
relevant_data = fetch_data_by_category_and_conditions(category, conditions)
print(relevant_data)

[{'ino': 1, 'installment_category': '복리/단리', 'installment_name': '퍼스트가계적금', 'installment_bank': '한국스탠다드차타드은행', 'installment_max_rate': '4.75', 'installment_normal_rate': '2.85', 'installment_join': '1', 'installment_stream': '36', 'installment_amount': '10000000', 'installment_way': '영업점,인터넷,스마트폰', 'installment_target': '개인(개인사업자 포함)'}, {'ino': 2, 'installment_category': '복리', 'installment_name': '행복적금', 'installment_bank': '신한은행', 'installment_max_rate': '4.50', 'installment_normal_rate': '2.70', 'installment_join': '1', 'installment_stream': '24', 'installment_amount': '5000000', 'installment_way': '영업점,인터넷', 'installment_target': '개인'}, {'ino': 4, 'installment_category': '복리', 'installment_name': '꿈나무적금', 'installment_bank': '기업은행', 'installment_max_rate': '4.20', 'installment_normal_rate': '3.00', 'installment_join': '1', 'installment_stream': '36', 'installment_amount': '20000000', 'installment_way': '인터넷,스마트폰', 'installment_target': '개인(개인사업자 포함)'}, {'ino': 6, 'installment_catego

In [42]:
print(category)
print(conditions)

적금
['복리']


In [44]:
import pandas as pd


# relevant_data를 DataFrame으로 변환
df_relevant_data = pd.DataFrame(relevant_data)

# DataFrame 출력
print(df_relevant_data)

    ino installment_category installment_name installment_bank  \
0     1                복리/단리          퍼스트가계적금      한국스탠다드차타드은행   
1     2                   복리             행복적금             신한은행   
2     4                   복리            꿈나무적금             기업은행   
3     6                   복리             미래적금             농협은행   
4     8                   복리            행복한적금             전북은행   
5    10                   복리             정성적금             제주은행   
6    12                   복리             튼튼적금           상호저축은행   
7    14                   복리             열정적금               신협   
8    16                   복리             희망적금             대구은행   
9    18                   복리            스마일적금             서민금융   
10   20                   복리           열정가득적금           상호저축은행   
11   22                   복리            퍼스트적금             농협은행   
12   24                   복리           미래가득적금             경남은행   
13   26                   복리            희망찬적금             전북은행   
14   28   

## 예시 4 (펀드)

In [47]:
# 사용자 쿼리 예시
query = "2000원으로 할 수 있는 펀드는 어떤게 있어?"

# 카테고리와 세부 조건을 추출
extracted_data = extract_categories(query)
category = extracted_data['category']
conditions = extracted_data['conditions']

# 카테고리와 조건에 맞는 데이터 추출
relevant_data = fetch_data_by_category_and_conditions(category, conditions)
print(relevant_data)

[{'fno': 1, 'fund_category': '수익률', 'fund_name': '삼성누버거버먼미국리츠부동산자투자신탁H[REITs-재간접형]_Ce', 'fund_earning_ratio': 17.58, 'fund_standard': 1158.26, 'fund_change_rate': -1.09, 'fund_scale': 258.68, 'fund_company': '삼성자산운용', 'fund_lisk': '높은위험', 'fund_type': '글로벌리츠재간접', 'fund_charge': '수수료 없음', 'fund_regdate': '2020/02/05'}, {'fno': 2, 'fund_category': '수익률', 'fund_name': 'NH-Amundi코리아2배인버스레버리지증권투자신탁[주식-파생재간접형]ClassCe', 'fund_earning_ratio': 15.57, 'fund_standard': 224.09, 'fund_change_rate': -11.57, 'fund_scale': 1531.36, 'fund_company': 'NH-Amundi자산운용', 'fund_lisk': '매우높은위험', 'fund_type': '베어마켓', 'fund_charge': '수수료 없음', 'fund_regdate': '2016/11/03'}, {'fno': 3, 'fund_category': '수익률', 'fund_name': 'NH-Amundi인도네시아포커스증권투자신탁[주식]ClassCe', 'fund_earning_ratio': 12.62, 'fund_standard': 960.05, 'fund_change_rate': 0.51, 'fund_scale': 218.99, 'fund_company': 'NH-Amundi자산운용', 'fund_lisk': '다소높은위험', 'fund_type': '기타국가주식', 'fund_charge': '수수료 없음', 'fund_regdate': '2008/01/11'}, {'fno': 4, 'fund_categ

In [48]:
print(category)
print(conditions)

펀드
['2000원']


In [49]:
import pandas as pd


# relevant_data를 DataFrame으로 변환
df_relevant_data = pd.DataFrame(relevant_data)

# DataFrame 출력
print(df_relevant_data)

    fno fund_category                                      fund_name  \
0     1           수익률            삼성누버거버먼미국리츠부동산자투자신탁H[REITs-재간접형]_Ce   
1     2           수익률  NH-Amundi코리아2배인버스레버리지증권투자신탁[주식-파생재간접형]ClassCe   
2     3           수익률             NH-Amundi인도네시아포커스증권투자신탁[주식]ClassCe   
3     4           수익률                  하나글로벌리츠부동산자투자신탁[재간접형]ClassC-E   
4     5           수익률                    하나글로벌인프라증권자투자신탁[주식]ClassC-E   
5     6           수익률                         삼성아세안증권자투자신탁 2[주식](Ce)   
6     7           수익률                  KB글로벌코어리츠부동산자투자신탁(재간접형)(H)C-E   
7     8           판매액                         KB머니마켓액티브증권투자신탁(채권)C-E   
8     9           판매액                        한화단기국공채증권자투자신탁(채권)종류C-e   
9    10           판매액                한국투자크레딧포커스ESG증권자투자신탁 1(채권)(C-e)   
10   11           판매액                           코레이트초단기금리혼합자산투자신탁_Ce   
11   12           판매액            한화2.2배레버리지인덱스증권투자신탁(주식-파생재간접형)종류C-e   
12   14           판매액                     KB스타단기국공채증권자투자신탁(채권)C-

## 예시 5 (외환)

In [58]:
# 사용자 쿼리 예시
query = "1000원으로 할 수 있는 환율 재테크는 어떤게 있어?"

# 카테고리와 세부 조건을 추출
extracted_data = extract_categories(query)
category = extracted_data['category']
conditions = extracted_data['conditions']

# 카테고리와 조건에 맞는 데이터 추출
relevant_data = fetch_data_by_category_and_conditions(category, conditions)
print(relevant_data)

[{'eno': 1, 'exchange_name': 'CNH', 'exchange_basic_rate': Decimal('187.96'), 'exchange_buy': Decimal('186.08'), 'exchange_sell': Decimal('189.83')}, {'eno': 4, 'exchange_name': 'JPY(100)', 'exchange_basic_rate': Decimal('945.74'), 'exchange_buy': Decimal('936.28'), 'exchange_sell': Decimal('955.19')}]


In [59]:
print(category)
print(conditions)

외환
['1000원']


In [60]:
import pandas as pd


# relevant_data를 DataFrame으로 변환
df_relevant_data = pd.DataFrame(relevant_data)

# DataFrame 출력
print(df_relevant_data)

   eno exchange_name exchange_basic_rate exchange_buy exchange_sell
0    1           CNH              187.96       186.08        189.83
1    4      JPY(100)              945.74       936.28        955.19


## 예시 6 (카테고리가 없을경우) 

In [11]:
# 카테고리와 조건에 맞는 데이터를 데이터베이스에서 추출
def fetch_data_by_category_and_conditions(category, conditions):
    connection = get_connection()
    cursor = connection.cursor(dictionary=True)

    if category:  # 카테고리가 주어진 경우
        # 카테고리별 테이블과 해당 컬럼 결정
        table = ""
        price_column = ""
        interest_column = ""

        if category.lower() == '주식':
            table = 'tbl_stock'
            price_column = 'stock_price'
        elif category.lower() == '펀드':
            table = 'tbl_fund'
            price_column = 'fund_standard'
        elif category.lower() == '예금':
            table = 'tbl_deposit'
            interest_column = 'deposit_category'
        elif category.lower() == '적금':
            table = 'tbl_installment'
            interest_column = 'installment_category'
        elif category.lower() == '외환':
            table = 'tbl_exchange'
            price_column = 'exchange_buy'

        # 조건 기반으로 쿼리 생성
        where_clauses = []
        params = []

        for condition in conditions:
            if "원" in condition:  # 금액 조건일 때
                amount = int(condition.replace("원", "").replace(",", "").strip())
                if price_column:
                    where_clauses.append(f"{price_column} <= %s")
                    params.append(amount)
            elif "단리" in condition or "복리" in condition:  # 이자 방식일 때
                if interest_column:
                    where_clauses.append(f"{interest_column} LIKE %s")
                    params.append(f"%{condition}%")

        # 조건이 있을 경우 WHERE 절 추가
        query = f"SELECT * FROM {table}"
        if where_clauses:
            query += " WHERE " + " AND ".join(where_clauses)
        
        cursor.execute(query, tuple(params))
        results = cursor.fetchall()

    else:  # 카테고리가 없는 경우, 모든 테이블에서 조건 체크
        all_results = []
        
        # 각 테이블에 대해 조건 검색
        tables = {
            'tbl_stock': 'stock_price',
            'tbl_fund': 'fund_standard',
            'tbl_deposit': 'deposit_category',
            'tbl_installment': 'installment_category',
            'tbl_exchange': 'exchange_buy'
        }

        for table, column in tables.items():
            where_clauses = []
            params = []
            
            for condition in conditions:
                if "원" in condition:  # 금액 조건일 때
                    amount = int(condition.replace("원", "").replace(",", "").strip())
                    where_clauses.append(f"{column} <= %s")
                    params.append(amount)
                elif "단리" in condition or "복리" in condition:  # 이자 방식일 때
                    where_clauses.append(f"{column} LIKE %s")
                    params.append(f"%{condition}%")
                    
            # 조건이 있을 경우 WHERE 절 추가
            query = f"SELECT * FROM {table}"
            if where_clauses:
                query += " WHERE " + " AND ".join(where_clauses)
            
            cursor.execute(query, tuple(params))
            results = cursor.fetchall()
            all_results.extend(results)  # 결과를 모아둡니다.

        results = all_results  # 모든 테이블에서 찾은 결과를 설정합니다.

    connection.close()
    return results


In [15]:
# 사용자 쿼리 예시
query = "100000원으로 할 수 있는 건 뭐가있어??"

# 카테고리와 세부 조건을 추출
extracted_data = extract_categories(query)
category = extracted_data['category']
conditions = extracted_data['conditions']

# 카테고리와 조건에 맞는 데이터 추출
relevant_data = fetch_data_by_category_and_conditions(category, conditions)
print(relevant_data)

[{'sno': 1, 'stock_symbol': '005930', 'stock_name': '삼성전자', 'stock_price': '65200', 'price_change_rate': '-1.66', 'stock_volume': '8175458'}, {'sno': 6, 'stock_symbol': '005935', 'stock_name': '삼성전자우', 'stock_price': '53000', 'price_change_rate': '-0.75', 'stock_volume': '622801'}, {'sno': 9, 'stock_symbol': '105560', 'stock_name': 'KB금융', 'stock_price': '80700', 'price_change_rate': '3.33', 'stock_volume': '587431'}, {'sno': 11, 'stock_symbol': '055550', 'stock_name': '신한지주', 'stock_price': '55400', 'price_change_rate': '2.78', 'stock_volume': '330104'}, {'sno': 17, 'stock_symbol': '032830', 'stock_name': '삼성생명', 'stock_price': '96400', 'price_change_rate': '0.21', 'stock_volume': '49934'}, {'sno': 19, 'stock_symbol': '138040', 'stock_name': '메리츠금융지주', 'stock_price': '92400', 'price_change_rate': '2.33', 'stock_volume': '70403'}, {'sno': 20, 'stock_symbol': '086790', 'stock_name': '하나금융지주', 'stock_price': '60800', 'price_change_rate': '4.29', 'stock_volume': '384681'}, {'sno': 25, 'st

In [16]:
print(category)
print(conditions)


['100000원']


In [17]:
import pandas as pd


# relevant_data를 DataFrame으로 변환
df_relevant_data = pd.DataFrame(relevant_data)

# DataFrame 출력
print(df_relevant_data)

     sno stock_symbol stock_name stock_price price_change_rate stock_volume  \
0    1.0       005930       삼성전자       65200             -1.66      8175458   
1    6.0       005935      삼성전자우       53000             -0.75       622801   
2    9.0       105560       KB금융       80700              3.33       587431   
3   11.0       055550       신한지주       55400              2.78       330104   
4   17.0       032830       삼성생명       96400              0.21        49934   
..   ...          ...        ...         ...               ...          ...   
91   NaN          NaN        NaN         NaN               NaN          NaN   
92   NaN          NaN        NaN         NaN               NaN          NaN   
93   NaN          NaN        NaN         NaN               NaN          NaN   
94   NaN          NaN        NaN         NaN               NaN          NaN   
95   NaN          NaN        NaN         NaN               NaN          NaN   

    fno fund_category fund_name  fund_earning_ratio